In [7]:
import time 
import requests, bs4
import re
import sys
import os
import datetime
from urllib.parse import urlparse

import pandas as pd
import numpy as np
from omegaconf import OmegaConf
import pandas_gbq

sys.path.append("../")
from src.scraping import ScrapingBase, ScrapingSponavi

%matplotlib inline
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 出力先
conf_dir = "../config"
conf_cli = OmegaConf.from_cli()
conf_exec = OmegaConf.load(os.path.join(conf_dir, "config_exec.yaml"))
conf_path = OmegaConf.load(os.path.join(conf_dir, "config_path.yaml"))
conf_url = OmegaConf.load(os.path.join(conf_dir, "config_url.yaml"))
conf_team = OmegaConf.load(os.path.join(conf_dir, "config_team.yaml"))
config = OmegaConf.merge(conf_cli, conf_exec, conf_path, conf_url, conf_team)

In [9]:
lake_dir = "../data/lake"
project_id = 'baseball-analytics-311804'

In [4]:
# 試合結果
df_lake_game = pd.read_csv(os.path.join(lake_dir, "lake_game.tsv"), sep="\t")

In [5]:
df_lake_game

,game_id,game_date,game_start_time,game_time,game_series,game_status,game_result,team_top_id,team_top_name,team_bottom_id,...,picher_lose_id,picher_save_id,starting_picher_bottom_id,starting_picher_top_id,audience_num,umpire_plate,umpire_first,umpire_second,umpire_third,exec_datetime
0,npb2021000096,2021-03-26,17:30,3時間36分,pennant_race,finish,top_win,npb5,阪神,npb2,...,npb1800033,npb1600013,npb1200062,npb1200076,"9,984人",深谷,本田,有隅,橋本,2021-04-06 02:33:17
1,npb2021000095,2021-03-26,17:45,3時間28分,pennant_race,finish,bottom_win,npb3,DeNA,npb1,...,npb1200049,NaN,npb1600081,npb1200041,"9,991人",牧田,山路,笠原,柳田,2021-04-06 02:33:17
2,npb2021000097,2021-03-26,18:00,2時間54分,pennant_race,finish,top_win,npb4,中日,npb6,...,npb1700070,npb1300066,npb1200055,npb1300057,"16,452人",土山,吉本,眞鍋,福家,2021-04-06 02:33:17
3,npb2021000098,2021-03-26,16:00,3時間40分,pennant_race,finish,bottom_win,npb8,日本ハム,npb376,...,npb1100099,NaN,npb1100099,npb500014,"14,559人",津川,村山,川口,敷田,2021-04-06 02:33:17
4,npb2021000099,2021-03-26,17:45,2時間30分,pennant_race,finish,bottom_win,npb11,オリックス,npb7,...,npb1600153,npb1200082,npb1600153,npb1400182,"9,599人",木内,秋村,原,西本,2021-04-06 02:33:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,npb2021000034,2021-03-09,13:00,2時間32分,pre_season_match,finish,bottom_win,npb9,ロッテ,npb376,...,npb1200094,npb2000124,npb1800052,npb700065,"1,114人",本田,長川,鈴木,山路,2021-04-06 09:28:29
71,npb2021000035,2021-03-09,13:00,2時間44分,pre_season_match,finish,top_win,npb7,西武,npb4,...,npb1800061,NaN,npb1800080,npb1800061,"1,439人",木内,市川,古賀,秋村,2021-04-06 09:28:29
72,npb2021000037,2021-03-09,13:00,3時間9分,pre_season_match,finish,bottom_win,npb6,広島,npb5,...,npb1600121,npb1300076,npb1600121,npb1900044,"5,131人",丹波,福家,土山,山本貴,2021-04-06 09:28:29
73,npb2021000036,2021-03-09,18:00,2時間46分,pre_season_match,finish,bottom_win,npb2,ヤクルト,npb11,...,npb2000025,npb1800101,npb1800143,npb1700109,"4,466人",飯塚,眞鍋,芦原,橘高,2021-04-06 09:28:29


In [11]:
sql = """
SELECT * FROM `bigquery-public-data.austin_311.311_service_requests` LIMIT 100
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)

In [12]:
df

,unique_key,complaint_type,complaint_description,owning_department,source,status,status_change_date,created_date,last_update_date,close_date,...,incident_zip,county,state_plane_x_coordinate,state_plane_y_coordinate,latitude,longitude,location,council_district_code,map_page,map_tile
0,19-00492617,ACCOYTE,Coyote Complaints,Animal Services Office,Phone,Closed,2020-01-02 12:10:10+00:00,2019-12-31 17:44:33+00:00,2020-01-02 12:10:10+00:00,2020-01-02 12:10:10+00:00,...,78733,TRAVIS,3080480.50158721,1.009220e+07,30.329131,-97.848433,"(30.32913087, -97.84843274)",NaN,552C,MD27
1,20-00080514,ROADMARK,Road Markings/Striping - Maintenance,Transportation,Phone,Closed,2020-07-14 15:22:18+00:00,2020-03-04 12:57:08+00:00,2020-07-14 15:22:19+00:00,2020-07-14 15:22:18+00:00,...,78712,TRAVIS,3114541.49714749,1.007791e+07,30.287718,-97.741555,"(30.28771801, -97.74155505)",9.0,585E,MJ24
2,19-00399072,COAACDD,Dangerous/Vicious Dog Investigation,Animal Services Office,Phone,Closed,2019-10-17 11:24:18+00:00,2019-10-16 19:53:21+00:00,2019-10-17 11:24:18+00:00,2019-10-17 11:24:18+00:00,...,78653,TRAVIS,3183598.80552451,1.010889e+07,30.368280,-97.520348,"(30.36828035, -97.52034774)",NaN,500Y,MV31
3,19-00159437,DRFLOODG,Flooding Current (Non-Emergency),Watershed Protection,Phone,Closed,2019-12-13 12:47:20+00:00,2019-05-03 19:50:28+00:00,2019-12-13 12:47:20+00:00,2019-12-13 12:47:20+00:00,...,78742,TRAVIS,3133106.04885254,1.006058e+07,30.238885,-97.684061,"(30.23888481, -97.68406084)",3.0,616Q,MM20
4,20-00351457,TRANSPOR,Traffic Engineering - General,Transportation,Phone,Closed,2020-08-26 15:32:43+00:00,2020-08-26 12:54:34+00:00,2020-08-26 15:32:43+00:00,2020-08-26 15:32:43+00:00,...,78613,WILLIAMSON,3091111.94183145,1.014562e+07,30.475337,-97.810904,"(30.4753365, -97.8109037)",6.0,403Y,MF40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20-00406670,TRAFFICC,Speed Management,Transportation,Phone,Closed,2020-11-20 10:36:12+00:00,2020-10-02 13:46:52+00:00,2020-11-20 10:36:12+00:00,2020-11-20 10:36:12+00:00,...,78742,TRAVIS,3136819.53074679,1.005565e+07,30.225078,-97.672678,"(30.22507805, -97.67267839)",2.0,646D,MM18
96,16-00068830,COAACDD,Dangerous/Vicious Dog Investigation,Animal Services Office,Phone,Closed,2016-03-30 09:26:38+00:00,2016-03-26 15:55:55+00:00,2016-03-30 08:24:20+00:00,2016-03-30 09:26:38+00:00,...,78725,TRAVIS,3139601.49995539,1.007677e+07,30.282963,-97.662267,"(30.2829628, -97.66226661)",1.0,587J,MN23
97,16-00130801,ACLOANIM,Loose Animal Not Dog,Animal Services Office,Phone,Closed,2016-06-03 18:10:37+00:00,2016-05-30 12:28:31+00:00,2016-06-03 17:10:38+00:00,2016-06-03 18:10:37+00:00,...,78734,TRAVIS,3050558.57228561,1.011302e+07,30.388165,-97.941835,"(30.38816525, -97.94183484)",NaN,490K,WY32
98,16-00108419,ACVEHICL,Animal In Vehicle,Animal Services Office,Phone,Closed,2016-05-07 15:20:34+00:00,2016-05-07 07:54:25+00:00,2016-05-07 14:17:55+00:00,2016-05-07 15:20:34+00:00,...,78736,TRAVIS,3068701.750835,1.005558e+07,30.229194,-97.888309,"(30.22919412, -97.8883092)",8.0,611Y,MB19
